# Experiments on Phylo HMMs

In [ ]:
%matplotlib inline
from tree_serialisation import load_tree
from data_simulation import generate_case, rate_sub_HKY, scale_branches_length

import numpy as np
import random
from copy import deepcopy

import matplotlib.pyplot as plt

from felsenstein import pruning
from viterbi_sumproduct import viterbi, sum_product

## First example a toy gene finder

In [ ]:
# SIMULATION PARAMETERS
tree_path = "tree.json"
number_of_nucleotids = 100
alphabet = ['A', 'C', 'T', 'G']
alphabetSize = len(alphabet)

nbState = 4
# transition matrix of the toy gene finder
A = np.zeros((nbState, nbState))
A[0, 1] = 1
A[1, 2] = 1
A[2, 3] = 0.33
A[2, 0] = 1 - A[2, 3]
A[3, 3] = 0.33  # 0.9999  # unrealistic ...
A[3, 0] = 1 - A[3, 3]

# state initial probability
b = np.array([0.25, 0.25, 0.26, 0.24])

animalNames = ["dog", "cat", "pig", "cow", "rat", "mouse", "baboon", "human"]
n_species = len(animalNames)
"""[...], such as the higher average rate of substitution and the greater
transition/transversion ratio, in noncoding and third-codon-position sites
than in firstand second- codon-position sites[...]"""

pi = np.zeros((nbState, alphabetSize))
# substitution rates for pi 0 and 1 are between 0 and 0.001
pi[0] = np.random.rand(alphabetSize) * 0.001
pi[1] = np.random.rand(alphabetSize) * 0.001
# but between 0 and 0.01 for pi 2 and 3
pi[2] = np.random.rand(alphabetSize) * 0.01
pi[3] = np.random.rand(alphabetSize) * 0.01
pi /= pi.sum(axis=1)[:, None]

# translation/transversion rate
kappa = np.array([2.3, 2.7, 4.3, 5.4])

In [ ]:
# load the phylogenetic model from JSON
tree = load_tree(tree_path)

def sub_tree(tree, list_of_species):
    """
    Prune the tree to keep only keep species in a list
    Args :
           - tree (dict)
           - list_of_species (int list)list of index
    """
    tree_cp = deepcopy(tree)
    def treat_node(node):
        children = tree[node]  # important for the recursion
        if children:  # propagate recirsively
            for child in children:
                treat_node(child["node"])

            for child in children:
                new_child = child["node"]
                if new_child not in list_of_species and not tree_cp[new_child]:
                    # we drop the child
                    tree_cp[node].remove(child)
                    # discard "dead leaf"
                    tree_cp.pop(new_child, None)
                           
    treat_node(max(tree_cp.keys()))
   
    # now merge useless intermediary node ie with only one son
    def merge_unary(node, ancestor_list):
        if(len(tree_cp[node])== 1):
            child = tree_cp[node][0]["node"]          
            # if the child has ancestor we can remove _node_
            if ancestor_list:
                branch_length = tree_cp[node][0]["branch"]
                parent = ancestor_list[0]
                siblings = tree_cp[parent]
                for sibling in siblings:
                    if sibling['node'] == node:
                        sibling['node'] = child
                        sibling['branch'] += branch_length
                # child ancestor are now exactly node's ancestor
                merge_unary(child, ancestor_list)
            # if the child has no grand parent but has children
            elif tree_cp[tree_cp[node][0]["node"]]:
                tree_cp[child][0]["branch"] += tree_cp[node][0]["branch"]
                tree_cp.pop(node, None)
                merge_unary(child, [node])
            tree_cp.pop(node, None)
        elif len(tree_cp[node])== 2:
            merge_unary(tree_cp[node][0]["node"], [node]+ ancestor_list)
            merge_unary(tree_cp[node][1]["node"], [node]+ ancestor_list)

    merge_unary(max(tree_cp.keys()), [])
    tree_rn = {}
    def rename(node):
        # rename nodes so that all indices are betwen 1 2*n -1
        children = tree_cp[node] 
        for child in children:
            child_node = child["node"]
            child["node"] = sorted(tree_cp.keys()).index(child_node)+1
            rename(child_node) # rename child first
        # now rename actual node 
        node_renamed = sorted(tree_cp.keys()).index(node)+1
        tree_rn[node_renamed] = tree_cp[node]

    rename(max(tree_cp.keys()))       
    return tree_rn

tree = sub_tree(tree, [1,2,5,6])
print(tree)

In [ ]:
tree = sub_tree(tree, [1,2,5,6])
print(tree)
trees = []

for j in range(nbState):
    trees.append(scale_branches_length(tree, scale=0.1 * random.random()*1.5))



strands, states = generate_case(A, b, pi, kappa,
                                trees, number_of_nucleotids)

In [ ]:
def convert_strands(strands):
    # Transform strands from ints to strings
    str_strands = list()
    for strand in strands:
        str_strand = ""
        for acid_int in strand:
            str_strand = ''.join([str_strand, alphabet[acid_int]])
        str_strands += [str_strand]
    # Transform strands in sites
    sites = list()
    for site_ind in range(number_of_nucleotids):
        sites += [''.join([str_strands[species_ind][site_ind] for species_ind in range(n_species)])]
    return sites

In [ ]:
# Process likelihoods with Felsenstein's algorithm
Qs = rate_sub_HKY(pi, kappa)
likelihoods = np.zeros((nbState, number_of_nucleotids))
for state in range(nbState):
    tree = trees[state]
    Q = Qs[state]
    p = pi[state]
    for site_ind, site in enumerate(sites):
        likelihoods[state, site_ind] = pruning(Q, p, tree, site)

In [ ]:
# VITERBI PARAMETERS
S = range(nbState)
state_sequence_viterbi = viterbi(S, A, b, likelihoods)

In [ ]:
# Precision
np.sum(states == state_sequence_viterbi) / number_of_nucleotids

In [ ]:
str_strands
sites

In [ ]:
sites